In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization, Add, MaxPooling2D, Flatten, Dense, Softmax, Activation, GlobalAveragePooling2D, ZeroPadding2D
from tensorflow.keras.optimizers import Adam, Nadam, SGD
import numpy as np
import h5py

In [2]:
print(tf.__version__)
tf.test.is_gpu_available()

2.9.1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
def dataGenerator(hdfile):
    def generator():
        indexs = np.arange(hdfile['Xs'].shape[0])
        for i in indexs:
            x = np.array(hdfile['Xs'][i])
            y = np.array(hdfile['Ys'][i] - 1)
            yield x, y
    return generator

In [4]:
hdf5 = h5py.File('./hdf5/play_style_dataset.hdf5','r')
hdf5_val = h5py.File('./hdf5/play_style_val.hdf5')
batch_size = 32
dataset_size = hdf5['Xs'].shape[0]
val_size = hdf5_val['Xs'].shape[0]

train_steps = int(dataset_size / batch_size)
val_steps = int(val_size / batch_size)
print(f"train_steps: {train_steps}, val_steps: {val_steps}")

data_gen = dataGenerator(hdf5)
dataset = tf.data.Dataset.from_generator(data_gen, 
                                         output_types=(tf.dtypes.float32, tf.dtypes.int32),
                                         output_shapes=(tf.TensorShape((19,19,28)),tf.TensorShape((1))))
dataset = dataset.batch(batch_size)
dataset = dataset.prefetch(tf.data.AUTOTUNE)

data_gen_valid = dataGenerator(hdf5_val)
dataset_valid = tf.data.Dataset.from_generator(data_gen_valid, 
                                         output_types=(tf.dtypes.float32, tf.dtypes.int32),
                                         output_shapes=(tf.TensorShape((19,19,28)),tf.TensorShape((1))))
dataset_valid = dataset_valid.batch(batch_size)
dataset_valid = dataset_valid.prefetch(tf.data.AUTOTUNE)

train_steps: 6653, val_steps: 41


In [5]:
# resnet50
def block1(x, filters, kernel_size=3, stride=1, conv_shortcut=False):
    preact = Activation("relu")(x)
    if conv_shortcut:
        shortcut = Conv2D(4 * filters, 1, strides=stride)(preact)
    else:
        shortcut = x
    x = Conv2D(filters, 1, strides=1)(preact)
    x = Activation("relu")(x)
    if kernel_size == 3:
        x = ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
    elif kernel_size == 5:
        x = ZeroPadding2D(padding=((2, 2), (2, 2)))(x)
    x = Conv2D(
        filters,
        kernel_size,
        strides=stride
    )(x)
    x = Activation("relu")(x)
    x = Conv2D(4 * filters, 1)(x)
    x = Add()([shortcut, x])
    return x

def ResNet50(include_top=True,
             input_tensor=None,
             input_shape=None,
             pooling=False):
    inputs = Input(shape = input_shape)
    x = block1(inputs, 256, kernel_size=3, conv_shortcut=True)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)
    x = block1(x, 256)#33
    
    x = GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dense(3, activation='softmax', name='fc1000')(x)

    model = Model(inputs, x, name='resnet50')
    return model

model = ResNet50(include_top=True,
                 input_tensor=None,
                 input_shape=(19, 19, 28),
                 pooling=False)
opt = Nadam(learning_rate = 0.0001)
model.compile(optimizer = opt,
              loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
model.summary()

In [6]:
callback1 = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                patience=5, verbose=0, mode='max'
                                 , restore_best_weights=False)
callback2 = tf.keras.callbacks.ModelCheckpoint('./models/model_style_v2_b32_f256_l100_{epoch:02d}.h5', 
                                             monitor='val_accuracy', 
                                            verbose=0, save_best_only=False, save_weights_only=False, 
                                            mode='max', save_freq="epoch")

In [7]:
# model =load_model('./models/model_style_v2_b32_f256_l100_aug16_1.h5')

In [ ]:
history = model.fit(
    dataset,
    epochs = 500,
    validation_data = dataset_valid,s
    callbacks = [callback1, callback2]
)

Epoch 1/500
     70/Unknown - 36s 292ms/step - loss: 1.1489 - accuracy: 0.3786

In [9]:
# model.save('./models/model_style_v2_b32_f256_l100_aug16_1.h5')